In [2]:
import pandas as pd
import seaborn as sns
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib
from sklearn.compose import make_column_transformer  # Importar correctamente

# Cargar el dataset y preprocesamiento inicial
df = sns.load_dataset('diamonds')
X = df.drop('cut', axis=1)
y = LabelEncoder().fit_transform(df['cut'])

# Definir las columnas numéricas y categóricas
numerical_cols = ['carat', 'depth', 'table', 'x', 'y', 'z']
categorical_cols = ['color', 'clarity']

# Pipeline para columnas numéricas y categóricas
preprocessor = make_column_transformer(
    (make_pipeline(SimpleImputer(strategy='median'), MinMaxScaler()), numerical_cols),
    (make_pipeline(SimpleImputer(strategy='most_frequent'), OneHotEncoder(sparse_output=False)), categorical_cols)
)

# Crear el pipeline completo con el clasificador RandomForest
pipeline = make_pipeline(
    preprocessor,
    RandomForestClassifier(random_state=42)
)

# División de los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenamiento del modelo
pipeline.fit(X_train, y_train)

# Predicción y evaluación
y_pred = pipeline.predict(X_test)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))
print("Matriz de confusión:\n", confusion_matrix(y_test, y_pred))

# Guardar el pipeline entrenado
joblib.dump(pipeline, '../models/pipeline_clasificacion.joblib')

# Predicción para nuevos datos
X_new = pd.DataFrame({
    'carat': [0.23],
    'color': ['E'],
    'clarity': ['SI2'],
    'depth': [61.5],
    'table': [55],
    'price': [326],
    'x': [3.95],
    'y': [3.98],
    'z': [2.43]
})
y_pred_new = pipeline.predict(X_new)
y_pred_label = LabelEncoder().fit(df['cut']).inverse_transform(y_pred_new)
print("Predicción de clasificación para el nuevo dato:", y_pred_label)

Accuracy: 0.7560
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.91      0.87      0.89       335
           1       0.74      0.65      0.69      1004
           2       0.82      0.91      0.86      4292
           3       0.73      0.80      0.76      2775
           4       0.62      0.46      0.52      2382

    accuracy                           0.76     10788
   macro avg       0.76      0.74      0.75     10788
weighted avg       0.75      0.76      0.75     10788

Matriz de confusión:
 [[ 292   29    3    7    4]
 [  24  655   27   56  242]
 [   2   11 3918  179  182]
 [   0   15  308 2207  245]
 [   4  175  540  579 1084]]
Predicción de clasificación para el nuevo dato: ['Ideal']
